This notebook hijacks the spyking-circus backbone to run a notch filter on your data.

# Step 0: set up filtering via spyking-circus modification in python

### First, you need to install the packages `mpi4py` and `blosc` in your environment.

`conda install -c conda-forge mpi4py`  
`conda install -c conda-forge python-blosc`  
`conda install -c conda-forge psutil`
  
Note that you might also need to install some other commonly installed packages (e.g. numpy) that aren't listed here.

Note that you might be able to accelerate everything with your GPU on Linux! See the following message that popped up when I installed `mpi4py`:  

```
For Linux 64, Open MPI is built with CUDA awareness but this support is disabled by default.
To enable it, please set the environment variable OMPI_MCA_opal_cuda_support=true before
launching your MPI processes. Equivalently, you can set the MCA parameter in the command line:
mpiexec --mca opal_cuda_support 1 ...
 
In addition, the UCX support is also built but disabled by default.
To enable it, first install UCX (conda install -c conda-forge ucx). Then, set the environment
variables OMPI_MCA_pml="ucx" OMPI_MCA_osc="ucx" before launching your MPI processes.
Equivalently, you can set the MCA parameters in the command line:
mpiexec --mca pml ucx --mca osc ucx ...
Note that you might also need to set UCX_MEMTYPE_CACHE=n for CUDA awareness via UCX.
Please consult UCX's documentation for detail.
```


### Next, you need to get the spyking-circus code that has been adjusted for notch filtering  
Either clone or copy files from here: https://github.com/nkinsky/spyking-circus.git

### Alternatively, you can install spyking-circus and then overwrite the `filtering.py` file with the one in my github repo above  
```
>> conda create -n circus python=3.6
>> conda activate circus  
(circus) >> conda install -c conda-forge -c spyking-circus spyking-circus
```
Full instructions here: https://spyking-circus.readthedocs.io/en/latest/introduction/install.html  
  
Then type: `$ conda env list` to find the spyking-circus phy environment base path (`circus_env_path`) and then navigate to `circus_env_path/lib/python3.6/site-packages/circus` or something similar to find the `filtering.py` file to overwrite.


# SUPER IMPORTANT NOTE:  
### Step #2 below will OVERWRITE any dat file you run it on, so make sure you have backup copies before proceeding.

After following the install steps at the bottom, run the following steps to de-noise the .dat file.

## Step 1: combine into one .dat file  
  
In terminal, type: `bash cat_dat_file.sh 104`  
This will create a new combined dat file named `continuous_combined.dat` while leaving the original dat files intact.

## Step 2: Remove EWL noise (and in some cases, 60 Hz noise) + harmonics  
  
Run the following code blocks:

In [ ]:
import sys
sys.path.append('/data/GitHub/spyking-circus')  # Add in the path to your spyking-circus repository here

from circus.filtering import miniscope_filter

dat_file = "/data/Working/Trace_FC/Recording_Rats/Finn/2022_01_17_habituation/continuous_combined.dat"
# This limits noise removal to the EWL frequency and its first three harmonics, and 60Hz + its
# harmonics up to top_limit
miniscope_filter(dat_filter, top_limit=300)  



## Step 2, Alternate #1 (No 60Hz noise):
In the event you don't have 60Hz noise (e.g. after 1/17 I used a headstage with GND and REF disconnected which eliminated 60Hz noise), you can just set top limit to 0 and adjust the notch_filter default params to eliminate 60Hz:

In [ ]:
notch_filter = [
            {"w0": 4843.0, "bw": 120, "Q": None},
            {"w0": 4843.0 * 2, "bw": 60, "Q": None},
            {"w0": 4843.0 * 3, "bw": 100, "Q": None},
        ]
miniscope_filter(dat_filter, top_limit=0)  

## Step 2, Alternate #2 (No 60Hz noise and drifting EWL noise during first recording):
In some cases, I have observed the EWL noise starting at 4849 Hz and then drifting to a steady state at ~4843.  You will need to adjust the default notch_filter parameters as follows for this case:


In [ ]:
notch_filter = [
            {"w0": 4843.0, "bw": 120, "Q": None},
            {"w0": 4849.0, "bw": 20, "Q": None},
            {"w0": 4843.0 * 2, "bw": 60, "Q": None},
            {"w0": 4843.0 * 3, "bw": 100, "Q": None},
        ]
miniscope_filter(dat_filter, top_limit=0)  

Just like for spyking-circus, you need to have a `session_name.dat` and `session_name.params` file in the same folder before running.  
  
~~At this point, you want to have the `overwrite` flag set to `False`, unlike for normal running of spyking-circus.~~  
NVM - seems to be lots of issues in spyking-circus with this.